In [ ]:
# k-nearest neighbors on the Iris Flowers Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename): #load dataset csv sesuai namanya
    dataset = list() #buat list kosong bernama dataset
    with open(filename, 'r') as file: #open file
        csv_reader = reader(file)
        for row in csv_reader: #baca data satu per satu dengan perulangan
            if not row: 
                continue
            dataset.append(row) #tambahkan data satu per satu ke dalam list dataset
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset: #untuk setiap baris data dalam dataset
        row[column] = float(row[column].strip()) #ubah nilai pada kolom data ke float, hilangkan spasinya

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset] #buat variabel class_values berisi data dari kolom terakhir di dataset
    unique = set(class_values) #buat himpunan data class_values bernama unique
    lookup = dict() #buat dictionary bernama lookup (dictionary adalah kumpulan data yg berpasangan seperti pada kamus indonesia inggris)
    for i, value in enumerate(unique): #perulangan sebanyak nilai pada variabel unique
        lookup[value] = i #tambahkan nilai i ke dalam dictionary lookup
    for row in dataset: #perulangan sebanyak jumlah baris dalam dataset
        row[column] = lookup[row[column]] #tambahkan nilai data ke dalam row
    return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list() #buat list bernama minmax
    for i in range(len(dataset[0])): #perulangan sejumlah panjang data yg pertama
        col_values = [row[i] for row in dataset] #buat variabel col_values, isi dengan nilai dari setiap baris data
        value_min = min(col_values) #simpan nilai data terkecil di value_min
        value_max = max(col_values) #simpan nilai data terbesar di value_max
        minmax.append([value_min, value_max]) #tambahkan nilai value_min dan value_max ke dalam variabel minmax secara berpasangan
    return minmax

# Minmax Scaling - Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0]) #ubah skala data menjadi 0 - 1 
      #skala diubah dengan rumus
      #data - nilai minimum / nilai maksimum - nilai minimum

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list() #membuat list kosong bernama dataset_split
    dataset_copy = list(dataset) #menyalin seluruh isi dataset ke dataset_copy
    fold_size = int(len(dataset) / n_folds) #variabel fold_size utk menampung ukuran fold sama jumlah data / n
    for _ in range(n_folds): #iterasi sejumlah n
        fold = list() #buat list kosong bernama fold
        while len(fold) < fold_size: #selama panjang fold < fold_size
            index = randrange(len(dataset_copy)) #buat angka random dari 0 hingga sebanyak data di dataset copy, masukkan ke index
            fold.append(dataset_copy.pop(index)) #tambahkan data dari dataset_copy ke dalam fold, sesuai index acak
        dataset_split.append(fold) #tambahkan data acak yg didapat ke dalam list dataset_split
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0 #buat variabel bernama correct untuk menghitung banyak prediksi yg benar
    for i in range(len(actual)): #perulangan sejumlah banyak prediksi yang benar
        if actual[i] == predicted[i]: #jika prediksi benar,
            correct += 1 #tambah hitungan correct
    return correct / float(len(actual)) * 100.0 #hitung persentase prediksi yang benar dalam persen

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds) #panggil fungsi cross_validation_split dengan parameter dataset dan jumlah n_folds
    scores = list() #buat variabel scores dengan tipe list
    for fold in folds: #perulangan sebanyak jumlah folds
        train_set = list(folds) #buat list untuk data training(train_set) sejumlah nilai folds
        train_set.remove(fold) #hapus nilai fold dari list (fold berisi angka 0 s.d folds)
        train_set = sum(train_set, []) #jumlahkan data trainset
        test_set = list() #buat list kosong untuk data test (test_set)
        for row in fold: #perulangan jumlah fold
            row_copy = list(row) #salin data ke row_copy
            test_set.append(row_copy) #tambahkan data dari row_copy ke test_set
            row_copy[-1] = None #buang data terakhir
        predicted = algorithm(train_set, test_set, *args) #ambil nilai hasil prediksi, simpan dalam variabel predicted 
        actual = [row[-1] for row in fold] #ambil hasil sebenarnya (bukan prediksi) dari dataset
        accuracy = accuracy_metric(actual, predicted) #hitung akurasi
        scores.append(accuracy) #tambahkan hasil perhitungan akurasi ke dalam scores
    return scores 

# Calculate the Euclidean distance between two vectors
#hitung jarak antara data training dengan data testing
def euclidean_distance(row1, row2):
    distance = 0.0 #buat variabel distance utk menyimpan jarak, beri nilai 0
    for i in range(len(row1)-1): #perulangan sebanyak jumlah data - 1
        distance += (row1[i] - row2[i])**2 #hitung jarak euclidean
    return sqrt(distance) #hitung akar kuadrat 

# Locate the most similar neighbors
# dapatkan data neighbors (yg paling mirip)
def get_neighbors(train, test_row, num_neighbors):
    distances = list() #buat list kosong bernama distance
    for train_row in train: #perulangan sebanyak train row 
        dist = euclidean_distance(test_row, train_row) #hitung jarak antara data test dengan data training, simpan hasilnya di dist
        distances.append((train_row, dist)) #tambahkan data training dan jaraknya ke dalam list distances
    distances.sort(key=lambda tup: tup[1]) #urutkan data dari kecil ke besar berdasarkan distance
    neighbors = list() #buat list kosong bernama neighbors
    for i in range(num_neighbors): #perulangan sebanyak jumlah neighbor
        neighbors.append(distances[i][0]) #tambahkan hasil perhitungan jarak ke dalam list neighbor 
    return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)

# Test the kNN on the Iris Flowers dataset
seed(1) #?
filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10 #data dibagi 5 sama rata
num_neighbors = 5 #nilai k
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0, 100.0, 100.0, 93.33333333333333, 93.33333333333333, 100.0, 100.0]
Mean Accuracy: 96.667%
